In [1]:
import timeit
from statistics import mean

In [2]:
def fib_naive(k):
    if k <= 1:
        return k
    return fib_naive(k - 1) + fib_naive(k - 2)

def fib_sequence_naive(n):
    return [fib_naive(i) for i in range(n)]


In [3]:
# Memoization (Top-Down DP)
def fib_memo(k, memo):
    if k in memo:
        return memo[k]
    if k <= 1:
        return k
    memo[k] = fib_memo(k - 1, memo) + fib_memo(k - 2, memo)
    return memo[k]

def fib_sequence_memo(n):
    memo = {}
    return [fib_memo(i, memo) for i in range(n)]

In [4]:
# Tabulation (Bottom-Up DP)
def fib_sequence_tab(n):
    if n == 0:
        return []
    if n == 1:
        return [0]
    dp = [0] * n
    dp[1] = 1
    for i in range(2, n):
        dp[i] = dp[i - 1] + dp[i - 2]
    return dp


In [5]:
# Timing helpers
def time_function(stmt_func, repeats=5, number=1):
    """
    Times stmt_func() using timeit. Returns average time in seconds.
    repeats: number of repeated measurements
    number: how many times per measurement (keep 1 for expensive funcs)
    """
    times = timeit.repeat(stmt_func, repeat=repeats, number=number)
    return mean(times)

def fmt_ms(seconds):
    return f"{seconds * 1000:.3f} ms"

def latex_escape(s):
    return s.replace("_", r"\_")


In [15]:
# Benchmark configurations
# Choose n values to test.
# Keep naive small (e.g., 10–30). DP methods can handle larger.
n_values = [5, 10, 15, 25, 50, 250, 500, 1000]

# Max n for naive (to avoid very long runs)
NAIVE_MAX_N = 40

# How many timing repeats
REPEATS = 10


In [16]:
# Run benchmarks
methods = [
    ("Naive recursion", fib_sequence_naive),
    ("Memoization (top-down)", fib_sequence_memo),
    ("Tabulation (bottom-up)", fib_sequence_tab),
]

results = []  # rows: (n, method, avg_seconds)

for n in n_values:
    for method_name, method_func in methods:
        if method_name == "Naive recursion" and n > NAIVE_MAX_N:
            results.append((n, method_name, None))
            continue

        avg_s = time_function(lambda n=n, f=method_func: f(n), repeats=REPEATS, number=1)
        results.append((n, method_name, avg_s))

In [17]:
# Print runtime comparison table (console)
# Build a pivot-like display: one row per n, columns = methods
method_order = [m[0] for m in methods]
header = ["n"] + method_order
print("\nRuntime comparison (average of {} runs)".format(REPEATS))
print("-" * 70)
print("{:>4} | {:>18} | {:>22} | {:>22}".format(*header))
print("-" * 70)

for n in n_values:
    row = [n]
    for m in method_order:
        t = next(val for (nn, mm, val) in results if nn == n and mm == m)
        row.append("SKIPPED" if t is None else fmt_ms(t))
    print("{:>4} | {:>18} | {:>22} | {:>22}".format(row[0], row[1], row[2], row[3]))

print("-" * 70)


Runtime comparison (average of 10 runs)
----------------------------------------------------------------------
   n |    Naive recursion | Memoization (top-down) | Tabulation (bottom-up)
----------------------------------------------------------------------
   5 |           0.002 ms |               0.002 ms |               0.001 ms
  10 |           0.016 ms |               0.005 ms |               0.001 ms
  15 |           0.171 ms |               0.006 ms |               0.001 ms
  25 |          23.177 ms |               0.009 ms |               0.002 ms
  50 |            SKIPPED |               0.016 ms |               0.003 ms
 250 |            SKIPPED |               0.084 ms |               0.014 ms
 500 |            SKIPPED |               0.191 ms |               0.034 ms
1000 |            SKIPPED |               0.392 ms |               0.077 ms
----------------------------------------------------------------------


In [18]:
# Print LaTeX table for Overleaf
print("\nLaTeX table (paste into Overleaf):\n")

latex_lines = []
latex_lines.append(r"\begin{table}[h]")
latex_lines.append(r"\centering")
latex_lines.append(r"\begin{tabular}{rlll}")
latex_lines.append(r"\hline")
latex_lines.append(r"$n$ & Naive recursion & Memoization (top-down) & Tabulation (bottom-up) \\")
latex_lines.append(r"\hline")

for n in n_values:
    cells = [str(n)]
    for m in method_order:
        t = next(val for (nn, mm, val) in results if nn == n and mm == m)
        cells.append(r"\textit{skipped}" if t is None else fmt_ms(t))
    latex_lines.append(" {} & {} & {} & {} \\\\".format(*cells))

latex_lines.append(r"\hline")
latex_lines.append(r"\end{tabular}")
latex_lines.append(r"\caption{Runtime comparison for generating the first $n$ Fibonacci numbers (average of " + str(REPEATS) + r" runs).}")
latex_lines.append(r"\label{tab:fib-runtime}")
latex_lines.append(r"\end{table}")

print("\n".join(latex_lines))


LaTeX table (paste into Overleaf):

\begin{table}[h]
\centering
\begin{tabular}{rlll}
\hline
$n$ & Naive recursion & Memoization (top-down) & Tabulation (bottom-up) \\
\hline
 5 & 0.002 ms & 0.002 ms & 0.001 ms \\
 10 & 0.016 ms & 0.005 ms & 0.001 ms \\
 15 & 0.171 ms & 0.006 ms & 0.001 ms \\
 25 & 23.177 ms & 0.009 ms & 0.002 ms \\
 50 & \textit{skipped} & 0.016 ms & 0.003 ms \\
 250 & \textit{skipped} & 0.084 ms & 0.014 ms \\
 500 & \textit{skipped} & 0.191 ms & 0.034 ms \\
 1000 & \textit{skipped} & 0.392 ms & 0.077 ms \\
\hline
\end{tabular}
\caption{Runtime comparison for generating the first $n$ Fibonacci numbers (average of 10 runs).}
\label{tab:fib-runtime}
\end{table}
